<a href="https://colab.research.google.com/github/momo54/large_scale_data_management/blob/main/PyPageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install -q findspark

In [ ]:
import findspark
findspark.init()

# SPARK INSTALLED... lets play

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
# !wget -q https://storage.googleapis.com/public_lddm_data/small_page_links.nt
# !ls

In [ ]:
!pip install rdflib

In [ ]:
with open("data/small_page_links.nt", "r") as file:
    lines = file.readlines()

split_lines = [line.strip().split() for line in lines]

In [ ]:
parsed_data = [(str(triple[0]), str(triple[1]), str(triple[2])) for triple in split_lines]
df = spark.createDataFrame(parsed_data, schema='subject string, predicate string, object string')

df.head(5)

In [ ]:
import re
def computeContribs(urls, rank) :
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)


def parseNeighbors(urls) :
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[2]

In [ ]:
from pyspark.sql import functions as F

# Loads all URLs from input file and initialize their neighbors.
links_df = df.select(df[0].alias("source"), df[2].alias("neighbor"))
links_df = links_df.distinct()
links_df = links_df.groupBy("source").agg(F.collect_list("neighbor").alias("neighbors"))

import time
start_time = time.time()

# Ajout du partitionnement par URL (exemple de partitionnement par 'source')
num_partitions = 4  # Définir le nombre de partitions souhaité
links_df = links_df.repartition(num_partitions, "source")

links_df = links_df.cache()

# Loads all URLs with other URL(s) link to from input file and initialize ranks of them to one.
ranks_df = links_df.select("source").distinct().withColumn("rank", F.lit(1.0))

In [ ]:
links_df.head(5)

In [ ]:
links_df = links_df.withColumn("count", F.size("neighbors"))
links_df = links_df.sort(F.col("count").desc())
links_df.select("source", "count").head(10)

In [ ]:
ranks_df.head(5)

In [ ]:
joined_df = links_df.join(ranks_df, on="source", how="inner")
joined_df.head(5)

In [ ]:
contrib = [(url, contrib) for row in joined_df.collect() for url, contrib in computeContribs(row["neighbors"], row["rank"])]
contrib_df = spark.createDataFrame(contrib, schema='source string, contrib float')

In [ ]:
contrib_df.head(5)

In [ ]:
for iteration in range(1):
  # Calculates URL contributions to the rank of other URLs.
  contrib = [(url, contrib) for row in joined_df.collect() for url, contrib in computeContribs(row["neighbors"], row["rank"])]
  contrib_df = spark.createDataFrame(contrib, schema='source string, contrib float')
  
  # Re-calculates URL ranks based on neighbor contributions.
  ranks_df = contrib_df.groupBy("source").agg(F.sum("contrib").alias("total_contrib"))
  ranks_df = ranks_df.withColumn("rank", ranks_df["total_contrib"] * 0.85 + 0.15)

  ranks_df = ranks_df.drop("total_contrib")
  
  # Collects all URL ranks and dump them to console.

end_time = time.time()  # Arrête le chronomètre
execution_time = end_time - start_time

for row in ranks_df.collect():
    print("%s has rank: %s." % (row["source"], row["rank"]))


In [ ]:
print(f"Temps d'exécution : {execution_time} secondes")